In [47]:
import numpy as np
import matplotlib.pyplot as plt

In [67]:
# Global Constants
DIRECTIONS          = np.array([(0, -1), (1, 0), (0, 1), (-1, 0)])   # UP, RIGHT, DOWN, LEFT
DEFAULT_TRAVEL_TIME = 99

# Calculate BOMB_MASK one time
COLS = ROWS        = 17
BLAST              = np.array([-3, -2, -1, 1, 2, 3])
BOMB_MASK = np.full((COLS, ROWS, COLS, ROWS), False)

x_inside = lambda x: x > 0 and x < COLS-1
y_inside = lambda y: y > 0 and y < ROWS-1

for x in range(1, COLS-1):
        for y in range(1, ROWS-1):
            if (x % 2 == 1 or y % 2 == 1):
                explosion_spots = [(x, y)]
                if x % 2 == 1:
                    explosion_spots += [(x, y + b) for b in BLAST  if y_inside(y + b)]
                if y % 2 == 1:
                    explosion_spots += [(x + b, y) for b in BLAST  if x_inside(x + b)]
                
                explosion_spots = tuple(np.array(explosion_spots).T)
                BOMB_MASK[(x, y)][explosion_spots] \
                                = True

In [49]:
def proximity_map (own_position, game_field):
    """
    calculates three values for each tile of the game field:
    1. travel time aka. distance from own position
    2. if tile is reachable from own position or blocked
    3. in which directions one can initially go to reach that tile as quick as possible

    Arguments
    ---------
    own_position : tuple (x, y)
        with x and y being current coordinates coordinates of the agent 
        on the game field. Thus 0 < x < COLS-1, 0 < y < ROWS-1.
    game_field   : np.array, shape = (COLS, ROWS)
        = game_state['field']

    Returns
    -------
    travel_time_map         : np.array, shape like game_field, dtype = int
        Reachable tiles have the value of the number of steps it takes to move to them 
        from own_position.
        Unreachable tiles have the value of DEFAULT_TRAVEL_TIME which is much higher than 
        any reachable time.
    reachable_map           : np.array, shape like game_field, dtype = bool
        A boolean mask of travel_time_map that labels reachable tiles as True and 
        unreachable ones as False.
    original_directions_map : np.array, shape = (COLS, ROWS, 4), dtype = bool
        A map of the game_field that holds a 4-element boolean array for every tile.
        Values of the tile's array correspond to the 4 directions UP, RIGHT, DOWN, LEFT 
        which you might from own_position to reach the tile. Those direction which lead you 
        to reach the tile the fastest are marked True, the others False.
        For example, if you can reach a tile the fastest by either going UP or RIGHT at the step
        then its array will look like this [TRUE, TRUE, FALSE, FALSE].
        This map will be important to quickly find the best direction towards coins, crates,
        opponents and more.
    """


    # Setup of initial values
    distance_map  = np.full_like(game_field, DEFAULT_TRAVEL_TIME)
    direction_map = np.full((*game_field.shape, 4), False)

    distance_map[own_position] = 0
    for i, dir in enumerate(DIRECTIONS):
        neighbor = tuple(dir + np.array(own_position))
        if game_field[neighbor] == 0:   # If neighbor is a free field
            direction_map[neighbor][i] = True
    

    # Breadth first search for proximity values to all reachable spots
    frontier = [own_position]
    while len(frontier) > 0:
        current = frontier.pop(0)
        
        for dir in DIRECTIONS:
            neighbor = tuple(dir + np.array(current))
            
            # Update travel time to `neighbor` field
            if game_field[neighbor] == 0:   # If neighbor is a free field
                time = distance_map[current] + 1
                if distance_map[neighbor] > time:
                    distance_map[neighbor] = time
                    frontier.append(neighbor)
                    
                    # Update original direction for `neighbor` field
                    if time > 1:
                        direction_map[neighbor] = direction_map[current]
                        
                # Combine orginial directions if travel times are equal
                elif distance_map[neighbor] == time:
                    direction_map[neighbor] = np.logical_or(
                        direction_map[neighbor], direction_map[current])


    # Derivation of reachability_map
    reachability_map = distance_map != DEFAULT_TRAVEL_TIME


    return distance_map, reachability_map, direction_map

In [162]:
def state_to_features (game_state):
    """
    """
    
    # 0. relevant game_state info
    own_position = game_state['self'][3]
    neighbors = own_position + DIRECTIONS

    crate_map    = game_state['field']
    collectable_coins = game_state['coins']
    bombs = game_state['bombs'] 
    explosion_map = game_state['explosion_map']


    # 1. Check game mode

    if collectable_coins != []: # only reachable ?
        f_6 = 0
    else:
        f_6 = 1
    
    '''
    for Task 3: 

    compute # coins

    elif # coins > 0:
        f_6 = 1

    elif game_state['others] != [] and # coins = 0:
        f_6 = 2
    '''

    features = [1,1,1,1,1,f_6] # UP, RIGHT, DOWN, LEFT, OWN_POSITION, MODUS

    # 2. Calculate proximity map
    distance_map, reachability_map, direction_map = proximity_map(own_position, crate_map)

    # 3. Check for invalid moves & lethal danger

    going_is_dump = np.array([( (not reachability_map[(x,y)]) or explosion_map[(x,y)] ) for [x,y] in neighbors])
    waiting_is_dump = False
    bombing_is_dump = False
    if not game_state['self'][2]: 
        bombing_is_dump = True

    for (bomb_position, bomb_timer) in bombs:
        steps_until_explosion = bomb_timer + 1

        if waiting_is_dump == False:
            no_future_explosion_mask = np.logical_not(BOMB_MASK[bomb_position])
            rescue_distances = distance_map[reachability_map & no_future_explosion_mask] # improve by including explosions
            minimal_rescue_distance = DEFAULT_TRAVEL_TIME if (rescue_distances.size == 0) else np.amin(rescue_distances)
            if steps_until_explosion <= minimal_rescue_distance:
                waiting_is_dump = True
                bombing_is_dump = True

        safe_directions = np.amax(direction_map[reachability_map & no_future_explosion_mask & (distance_map <= steps_until_explosion)], axis = 0, initial = False)
        going_is_dump[np.logical_not(safe_directions)] = True

    if bombing_is_dump == False:
        no_future_explosion_mask = np.logical_not(BOMB_MASK[own_position])
        rescue_distances = distance_map[reachability_map & no_future_explosion_mask] # improve by including explosions
        minimal_rescue_distance = DEFAULT_TRAVEL_TIME if (rescue_distances.size == 0) else np.amin(rescue_distances) 
        if minimal_rescue_distance >= 4:
                bombing_is_dump = True

    # 4. Compute goal direction
    goals = []

    # 5. Assemble feature array

    if waiting_is_dump:
        features[4] = 0

    elif own_position in goals:
        features[4] = 2

    for n, neighbor in enumerate(neighbors):
        if going_is_dump[n]:
            features[n] = 0
        
        elif neighbor in goals:
            features[n] = 2

    return(features)

x = 8

example_game_state = np.load(f'../../agent_code/our_user_agent/logs/game_states/game_state_{x}.pkl', allow_pickle = True)

%timeit state_to_features(example_game_state)

example_game_state['self'] = ('name', 0, True, (14,15))
print(f'Features: {state_to_features(example_game_state)} (UP, RIGHT, DOWN, LEFT, OWN_POSITION, MODUS)')

own_position = example_game_state['self'][3]
crate_map = example_game_state['field']
collectable_coins = example_game_state['coins']
bombs = example_game_state['bombs'] 
explosion_map = example_game_state['explosion_map']

crate_map[own_position] = 8
for bomb in bombs:
    crate_map[bomb[0]] += 3

print(f"Bomb action possible? {example_game_state['self'][2]}.")
print(crate_map.T) # transpose for image coordinates
print(explosion_map.T)


250 µs ± 5.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Features: [0, 0, 0, 1, 1, 1] (UP, RIGHT, DOWN, LEFT, OWN_POSITION, MODUS)
Bomb action possible? True.
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  1  1  1  0  1  1  1  1  0  0  1  0  0 -1]
 [-1  0 -1  0 -1  0 -1  0 -1  0 -1  1 -1  0 -1  0 -1]
 [-1  1  1  0  1  1  1  0  1  1  0  1  1  1  1  1 -1]
 [-1  1 -1  0 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1]
 [-1  1  0  1  1  1  1  1  0  0  1  1  1  0  0  0 -1]
 [-1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  0 -1]
 [-1  1  1  0  1  1  1  1  1  0  1  1  0  0  1  0 -1]
 [-1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1]
 [-1  0  1  1  1  1  0  1  1  1  0  1  1  1  1  1 -1]
 [-1  1 -1  0 -1  1 -1  1 -1  1 -1  1 -1  1 -1  0 -1]
 [-1  1  1  1  0  0  0  1  1  1  1  1  1  1  0  0 -1]
 [-1  1 -1  0 -1  1 -1  0 -1  1 -1  0 -1  1 -1  0 -1]
 [-1  1  1  1  0  1  0  0  0  1  1  1  0  0  0  0 -1]
 [-1  0 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  0 -1]
 [-1  0  0  1  

In [55]:
mask = np.array([[1,0], [0,1]])
print(mask[(0,1)])


0


array([[1, 2],
       [0, 0]])

In [181]:
np.zeros((15,3,3,6))[1,2,2]

array([0., 0., 0., 0., 0., 0.])